In [1]:
import pandas as pd
import json
import ast
import warnings
from io import StringIO
warnings.filterwarnings('ignore')

In [2]:
URL_DS_OUTPUT_STEAM = 'datasets/output_steam_games.json'

In [6]:
df_games = pd.read_json(URL_DS_OUTPUT_STEAM, lines=True)

In [17]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 12.9+ MB


In [22]:
df_games = df_games[df_games.isna().sum(axis=1) != 13]

In [25]:
df_games = df_games.reset_index()

In [26]:
df_games[df_games['id'] == 681550]

,index,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
32134,120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [31]:
df_games[df_games['app_name'] == 'Franchise Hockey Manager 2']

,index,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


In [28]:
df_games.publisher.value_counts()

publisher
Ubisoft                    385
Dovetail Games - Trains    360
Degica                     286
Paradox Interactive        271
SEGA                       251
                          ... 
MB² Entertainment            1
Back Of Nowhere Studios      1
Fourfold Games               1
Atomic Wolf                  1
Bidoniera Games              1
Name: count, Length: 8239, dtype: int64

In [5]:
lines_games = []
json_lines_games = []

with open(URL_DS_OUTPUT_STEAM, 'r', encoding='utf-8') as f_items:
    for line in f_items:
        lines_games.append(line)

# for i in lines_games:
    # json_lines_games.append(json.dumps(ast.literal_eval(i)))
    
lines_games

['{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}\n',
 '{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}\n',
 '{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}\n',
 '{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs": NaN, "price": NaN, "early_access": NaN, "id": NaN, "developer": NaN}\n',
 '{"publisher": NaN, "genres": NaN, "app_name": NaN, "title": NaN, "url": NaN, "release_date": NaN, "tags": NaN, "reviews_url": NaN, "specs"